In [1]:
# encoding=utf-8
# @playerYixin

# Mac: comment multiple lines = command + /
# Target: Read and Analyze local data from RSS Feed Explorer App - NetNewsWire
# --------
# 1: Find the file path where NetNewsWire's local data is stored.
# According to NetNewsWire's GitHub doc (https://github.com/Ranchero-Software/NetNewsWire/blob/main/Technotes/Accounts.markdown),
# the path should be something like "~/Library/Application Support/NetNewsWire/Accounts/" On the Mac.
# But it took me almost 6 hours to find the real one:
# "/Users/{userName}/Library/Containers/com.ranchero.NetNewsWire-Evergreen/Data/Library/Application Support/NetNewsWire/Accounts/{accountName}/". FYI.
# --------
# 2: Target data file is "DB.sqlite3", which contains 10 tables: articles, statuses, authors, authorsLookup, search, search_content, search_docsize, search_segdir, search_segments, search_stat
# Table "articles" contains each piece of article pushed by subscribed feeds, which new rows appended according to your refresh frequency.
# Column names are: articleID, feedID, uniqueID, title, contentHTML, contentText, url, external URL, summary, imageURL, bannerImageURL, datePublished, searchRowID.
# Table "statuses" contains tags (0/1) about read and starred articles.
# Column names are: articleID, read, starred, dateArrived
# other tables are about your searching behaviors and are not considerred in this task.
# --------
# 3: The original meta-data of your subscribed RSS feeds ("FeedMetadata.plist") can be found in the same path as "DB.sqlite3".
# You may need it to categorize and filter the articles.
# --------
# 4: This code provides a simplest implementation to achieve my infoShop project's target. (https://github.com/playerYixin/infoShop)
# You can modify it according to your taste of the information source.

import sqlite3
import pandas as pd
import datetime
#import dateutil.tz as tz
import plistlib
import markdown
import os

In [2]:
# Get timestamp range of 24 hours (between two successive UTC+0) before {prev} day

def getTimeRange(prev=0):
    dtEnd = datetime.datetime.combine(datetime.date.today()-datetime.timedelta(prev), datetime.datetime.min.time())
    tsEnd = dtEnd.replace(tzinfo=datetime.timezone.utc).timestamp()
    # tsEnd = dtEnd.replace(tzinfo=tz.tzlocal()).timestamp()
    dtStart = dtEnd - datetime.timedelta(1)
    tsStart = dtStart.replace(tzinfo=datetime.timezone.utc).timestamp()
    # tsStart = dtStart.replace(tzinfo=tz.tzlocal()).timestamp()
    return int(tsStart), int(tsEnd)

In [3]:
# Return timestamp range in consideration

PREV_DAYS = 0         # you can fetch ealier articles by modifying this parameter (int)

dtRange = getTimeRange(PREV_DAYS)
TODAY_TAG = datetime.datetime.fromtimestamp(dtRange[0]).strftime("%Y_%m_%d")
print("start timestamp: %s" % dtRange[0])
print("end timestamp: %s" % dtRange[1])
print("TODAY_TAG: %s" % TODAY_TAG)

start timestamp: 1665100800
end timestamp: 1665187200
TODAY_TAG: 2022_10_07


In [4]:
# Create a SQL connection to our SQLite database

# # set path_root of database
# path_root = "./"

# use the local path of NetNewsWire's local database
path_root = "/Users/{userName}/Library/Containers/com.ranchero.NetNewsWire-Evergreen/Data/Library/Application Support/NetNewsWire/Accounts/2_iCloud/"

# connect to database DB.sqlite3
con = sqlite3.connect(path_root + "DB.sqlite3")

# set encoding rule especially when the data includs Chinese characters.
cur = con.cursor()
cur.execute('pragma encoding=UTF8')

# # Use "cursor.execute" to fetch a single row, can be iterated over by row
# row = cur.execute('SELECT title FROM articles LIMIT 1;')
# row = row.fetchall()

# Use pandas.read_sql_query to fetch the whole table
# Do not include column named "contentHTML" to avoid pandas parsing it incorrectly
# column named "searchRowID" is also omitted here
column_select = "articleID, feedID, uniqueID, title, contentText, url, externalURL, summary, imageURL, datePublished"

df_article = pd.read_sql_query("SELECT %s FROM articles WHERE datePublished >= %s AND datePublished < %s ORDER BY datePublished DESC " % (column_select,dtRange[0],dtRange[1]), con, parse_dates=['datePublished'])

df_statuses = pd.read_sql_query("SELECT * FROM statuses WHERE dateArrived >= %s AND dateArrived < %s ORDER BY dateArrived DESC" % (dtRange[0],dtRange[1]), con, parse_dates=['dateArrived'])

# Be sure to close the connection
con.close()



In [5]:
len(df_article)

157

In [6]:
len(df_statuses)

122

In [7]:
# merge Table article and Table statuses on their common column named "articleID", print the length of the result

df_merge = df_article.merge(df_statuses,left_on='articleID', right_on='articleID',how='left')

print(df_merge.head(1).T)
print("\nLength of aticles: %s" % len(df_article))
print("Length of statuses: %s" % len(df_statuses))
print("Length of merged result: %s" % len(df_merge))

                                                               0
articleID                       0ef8a5abf01b9da9e919aeb57706d2aa
feedID                             https://cointelegraph.com/rss
uniqueID       https://cointelegraph.com/news/why-the-us-is-o...
title          Why the US is one of the most crypto-friendly ...
contentText                                                 None
url                                                         None
externalURL    https://cointelegraph.com/news/why-the-us-is-o...
summary                                                     None
imageURL                                                    None
datePublished                                2022-10-07 23:52:58
read                                                         NaN
starred                                                      NaN
dateArrived                                                  NaT

Length of aticles: 157
Length of statuses: 122
Length of merged result: 157


In [8]:
# if column 'externalURL' is NaN, fill it using column 'url''s value in the same row

df_merge['externalURL'] = df_merge[['url','externalURL']].fillna(method="ffill",axis=1)['externalURL']

In [9]:
# create and set output path

path_output = "./output"
isExist = os.path.exists(path_output)

if not isExist:
    # Create a new directory because it does not exist 
    os.makedirs(path_output)

In [10]:
# save result to .csv file (optional)

df_merge.to_csv('%s/NetNewsWire_DB_%s.csv' % (path_output, TODAY_TAG),encoding='utf_8_sig',index=False)

In [11]:
# Now you get the raw data from NetNewsWire's local DB. You can start analyze it by yourself.
# Below I share an example how I process the information.

In [12]:
# Before analyzing the articles, you'd better select your favorite feeds and add some tags to these sources.
# So that you can select articles from these feeds and read them in the first place.
# ~~~~~~~~
# In my case, I selected my favorite feeds and catergorized them into 6 groups:
# ---
# myAnalyst
# myProject
# myMedia
# myPodcasts
# myTracer
# myKOL
# ---
# I also add tags "mustRead" and "readTitlesOnly" to some of these selected feeds to remind me how to treat them according to my own reading strategy.
# ~~~~~~~~
# As a result, you will get a cleaned table of feeds' mata data
# You can get the raw feedMetaData from NetNewsWire's local file "FeedMetadata.plist" and edit based on that.

In [13]:
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# # This step show how I create my tagged feedMetaData file.
# # If you have already created a "feedMetaDataTagged.csv" file, and NO NEW feeds are subscribed afterwords, skip this
# # Anytime you add a new feed to NetNewsWire, you need to append it to the table.
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# # 1.fetch unique list of subscribed feeds
# with open(path_root + "FeedMetadata.plist", 'rb') as fp:
#     pl = plistlib.load(fp)
# feedMetaData = pd.DataFrame.from_dict(pl,orient='index')
# feedMetaData.reset_index(inplace=True)

# feedList = feedMetaData['feedID'].tolist()
# print("Total number of feeds: %s" % len(feedList))

# # 2.save latest feedMetaData 
# feedMetaData.to_csv('./feedMetaData_%s.csv' % TODAY_TAG,encoding='utf_8_sig',index=False)

# # 3.1(First, Manually) Add tags to feeds and save as a new file named "feedMetaDataTagged.csv"
# # 3.2(Manually) Copy information related to the new feeds to "feedMetaDataTagged.csv" and add tags

In [14]:
# load tagged feeds' meta data
df_feed = pd.read_csv('./feedMetaDataTagged.csv')

# fetch unique list of subscribed feeds
with open(path_root + "FeedMetadata.plist", 'rb') as fp:
    pl = plistlib.load(fp)
feedMetaData = pd.DataFrame.from_dict(pl,orient='index')
feedMetaData.reset_index(inplace=True)

# check if there's any new feed not recorded in "feedMetaDataTagged.csv"
feed_diff = list(set(feedMetaData['feedID'].to_list()).difference(set(df_feed['feedID'].to_list())))
if len(feed_diff)==0:
    print('There no missing feed.\n')
else:
    print('Missing feed(s):')
    print(feed_diff)
    print('\n')

# get Manual-added tag list except for 'mustRead' and 'readTitleOnly'
feedTag = list(set(list(df_feed.columns)).difference(set(list(feedMetaData.columns)+['mustRead','readTitleOnly'])))
print('The personal tags are:\n%s' % feedTag)

Missing feed(s):
['https://www.blocknative.com/blog/tag/defi/rss.xml']


The personal tags are:
['myKOL', 'myProject', 'myAnalyst', 'myPodcast', 'myTracer', 'myMedia']


In [15]:
# merge article information and feed information into one table

df_merge2 = df_merge.merge(df_feed,left_on='feedID', right_on='feedID',how='left')
df_merge2.head(1).T

,0
articleID,0ef8a5abf01b9da9e919aeb57706d2aa
feedID,https://cointelegraph.com/rss
uniqueID,https://cointelegraph.com/news/why-the-us-is-o...
title,Why the US is one of the most crypto-friendly ...
contentText,None
url,None
externalURL,https://cointelegraph.com/news/why-the-us-is-o...
summary,None
imageURL,None
datePublished,2022-10-07 23:52:58


In [16]:
# Prepare for digesting articles' information
idx_list = []
for ii in range(len(feedTag)):
    idx_list.append(df_merge2[feedTag[ii]]==1)


In [17]:
# classify articles according to tags and save to csv files
columns_select = ['title','feedID','editedName','homePageURL','contentText','url','externalURL','datePublished','mustRead','readTitleOnly']

for ii in range(len(feedTag)):
    df_merge2.loc[idx_list[ii], columns_select].to_csv("%s/articles_%s_%s.csv" % (path_output, feedTag[ii], TODAY_TAG), encoding='utf_8_sig',index=False)

In [18]:
# Set up the format of markdown file. Here emoji '&#x1F4CC;'(pin) means mustRead; emoji '&#x26A1; (flash) means readTitleOnly
def writeRows(row, f):
    mark1 = '&#x1F4CC;' if row['mustRead']==1 else ''
    mark2 = '&#x26A1;' if row['readTitleOnly']==1 else ''
    f.write("* {}{}[{}]({})&nbsp;*from*&nbsp;***{}***\n".format(mark1, mark2,row['title'],row['externalURL'],row['editedName']).encode('utf-8'))
    return None

In [21]:
# Generate the newsletter in markdown format
NA_TAG = '*(N.A.)*'     # In case there's no article in certain category.

with open('%s/Daily Reading List_%s.md' % (path_output, TODAY_TAG), 'bw+') as f:
    
    f.write('## {}_{}\n'.format('Daily Reading List',TODAY_TAG).encode('utf-8'))
    
    tag = 'myTracer'
    f.write('### {}\n'.format('Industry Tracer').encode('utf-8'))
    df_tmp = pd.read_csv("%s/articles_%s_%s.csv" % (path_output, tag, TODAY_TAG)).sort_values(by=['mustRead','readTitleOnly'],ascending=False)
    rows = df_tmp.to_dict('records')
    if len(rows) == 0:
        f.write('{}\n'.format(NA_TAG).encode('utf-8'))
    else:
        for row in rows:
            writeRows(row, f)
        
    tag = 'myAnalyst'
    f.write('### {}\n'.format('Deep Dive').encode('utf-8'))
    df_tmp = pd.read_csv("%s/articles_%s_%s.csv" % (path_output, tag, TODAY_TAG)).sort_values(by=['mustRead','readTitleOnly'],ascending=False)
    rows = df_tmp.to_dict('records')
    if len(rows) == 0:
        f.write('{}\n'.format(NA_TAG).encode('utf-8'))
    else:
        for row in rows:
            writeRows(row, f)

    tag = 'myPodcast'
    f.write('### {}\n'.format('Podcast').encode('utf-8'))
    df_tmp = pd.read_csv("%s/articles_%s_%s.csv" % (path_output, tag, TODAY_TAG)).sort_values(by=['mustRead','readTitleOnly'],ascending=False)
    rows = df_tmp.to_dict('records')
    if len(rows) == 0:
        f.write('{}\n'.format(NA_TAG).encode('utf-8'))
    else:
        for row in rows:
            writeRows(row, f)
            
    tag = 'myProject'
    f.write('### {}\n'.format('Project Tracer').encode('utf-8'))
    df_tmp = pd.read_csv("%s/articles_%s_%s.csv" % (path_output, tag, TODAY_TAG)).sort_values(by=['mustRead','readTitleOnly'],ascending=False)
    rows = df_tmp.to_dict('records')
    if len(rows) == 0:
        f.write('{}\n'.format(NA_TAG).encode('utf-8'))
    else:    
        for row in rows:
            writeRows(row, f)
            
    tag = 'myKOL'
    f.write('### {}\n'.format("KOL's Opinion").encode('utf-8'))
    df_tmp = pd.read_csv("%s/articles_%s_%s.csv" % (path_output, tag, TODAY_TAG)).sort_values(by=['mustRead','readTitleOnly'],ascending=False)
    rows = df_tmp.to_dict('records')
    if len(rows) == 0:
        f.write('{}\n'.format(NA_TAG).encode('utf-8'))
    else:
        for row in rows:
            writeRows(row, f)
            
    f.seek(0)
    markdown.markdownFromFile(input=f, output='%s/Daily Reading List_%s.html' % (path_output, TODAY_TAG))